In [5]:
import pickle
import os
import pandas as pd

In [3]:
os.chdir("/orfeo/scratch/dssc/zenocosini/mmlu_result/transposed_dataset")

In [14]:
label_clustering_dataset_1 = pd.read_pickle("metrics_421512/label_clustering_dataset.pkl")
label_clustering_dataset_2 = pd.read_pickle("metrics_422088/label_clustering_dataset.pkl")

In [15]:
label_clustering_dataset = pd.concat([label_clustering_dataset_1,label_clustering_dataset_2])
label_clustering_dataset.to_pickle("shuffled_prompt/label_clustering_dataset.pkl")

In [16]:
label_clustering_only_ref_pred_1 = pd.read_pickle("metrics_421512/label_clustering_only_ref_pred.pkl")
label_clustering_only_ref_pred_2 = pd.read_pickle("metrics_422088/label_clustering_only_ref_pred.pkl")
label_clustering_only_ref_pred = pd.concat([label_clustering_only_ref_pred_1,label_clustering_only_ref_pred_2])
label_clustering_only_ref_pred.to_pickle("shuffled_prompt/label_clustering_only_ref_pred.pkl")

In [17]:
label_overlap_only_ref_pred_1 = pd.read_pickle("metrics_421512/label_overlap_only_ref_pred.pkl")
label_overlap_only_ref_pred_2 = pd.read_pickle("metrics_422088/label_overlap_only_ref_pred.pkl")
label_overlap_only_ref_pred = pd.concat([label_overlap_only_ref_pred_1,label_overlap_only_ref_pred_2])
label_overlap_only_ref_pred.to_pickle("shuffled_prompt/label_overlap_only_ref_pred.pkl")

In [18]:
label_overlap_dataset_1 = pd.read_pickle("metrics_421512/label_overlap_dataset.pkl")
label_overlap_dataset_2 = pd.read_pickle("metrics_422088/label_overlap_dataset.pkl")
label_overlap_dataset = pd.concat([label_overlap_dataset_1,label_overlap_dataset_2])
label_overlap_dataset.to_pickle("shuffled_prompt/label_overlap_dataset.pkl")

In [19]:
intrinsic_dimension_1 = pd.read_pickle("metrics_421512/intrinsic_dimension.pkl")
intrinsic_dimension_2 = pd.read_pickle("metrics_422088/intrinsic_dimension.pkl")
intrinsic_dimension = pd.concat([intrinsic_dimension_1,intrinsic_dimension_2])
intrinsic_dimension.to_pickle("shuffled_prompt/intrinsic_dimension.pkl")

In [24]:
!pwd

/orfeo/scratch/dssc/zenocosini/mmlu_result/transposed_dataset
